In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
import math
import sympy as sym
import time

priorsP = pd.DataFrame(columns = ['mass','sma','ecc','aop','inc','lan','mea','j2r2','c22r2','spaop','spinc','splan'],index=['dist. shape','uni-low','uni-up','log-uni-low','log-uni-up','norm-cen','norm-spread','log-norm-cen','log-norm-spread'])
paramsdf = pd.DataFrame(columns = ['name','mass','sma','ecc','aop','inc','lan','mea','j2r2','c22r2','spaop','spinc','splan'],index=['theKBO','obj1','obj2'])

paramsdf['name'] = ['Haumea','Hi\'aka','Namaka']
paramsdf['mass'] = [4.006*10**21,1.79*10*19,1.79*10**18]
paramsdf['sma'] = [np.NaN,49880,25657]
paramsdf['ecc'] = [np.NaN,0.0513,0.249]
paramsdf['aop'] = [np.NaN,154.1,178.9]
paramsdf['inc'] = [np.NaN,126.356,113.013]
paramsdf['lan'] = [np.NaN,206.766,205.016]
paramsdf['mea'] = [np.NaN,152.8,178.5]
paramsdf['j2r2'] = [np.NaN,4,5]
paramsdf['c22r2'] = [np.NaN,4,6]
paramsdf['spaop'] = [np.NaN,4,5]
paramsdf['spinc'] = [np.NaN,2,1]
paramsdf['splan'] = [np.NaN,4,6]

priorsP['mass'] = [2,0,10,1,2,1,1,4.006*10**21,0.040*10*20]
priorsP['sma'] = [2,0,12,1,2,35000,20000,1,1]
priorsP['ecc'] = [2,0,1,0,1,1,1,1,1]
priorsP['aop'] = [2,0,180,1,2,1,1,1,1]
priorsP['inc'] = [2,0,180,1,2,1,1,1,1]
priorsP['lan'] = [2,0,360,0,12,1,1,1,1]
priorsP['mea'] = [2,0,180,1,2,1,1,1,1]
priorsP['j2r2'] = [2,0,12,1,2,1,1,1,1]
priorsP['c22r2'] = [2,0,12,1,8,1,1,1,1]
priorsP['spaop'] = [2,0,16,1,6,1,1,1,1]
priorsP['spinc'] = [2,0,12,0,8,1,1,1,1]
priorsP['splan'] = [2,0,12,1,8,1,1,1,1]

priors1 = pd.DataFrame(columns = ['mass','sma','ecc','aop','inc','lan','mea','j2r2','c22r2','spaop','spinc','splan'],index=['dist. shape','uni-low','uni-up','log-uni-low','log-uni-up','norm-cen','norm-spread','log-norm-cen','log-norm-spread'])
priors2 = pd.DataFrame(columns = ['mass','sma','ecc','aop','inc','lan','mea','j2r2','c22r2','spaop','spinc','splan'],index=['dist. shape','uni-low','uni-up','log-uni-low','log-uni-up','norm-cen','norm-spread','log-norm-cen','log-norm-spread'])

priors1['mass'] = [3,0,10,1,2,1,1,1.79*10**19,0.11*10**19]
priors1['sma'] = [2,0,12,1,2,49880,198,1,1]
priors1['ecc'] = [2,0,1,0,1,1,1,1,1]
priors1['aop'] = [2,0,180,1,2,1,1,1,1]
priors1['inc'] = [2,0,180,1,2,1,1,1,1]
priors1['lan'] = [2,0,360,0,12,1,1,1,1]
priors1['mea'] = [2,0,180,1,2,1,1,1,1]
priors1['j2r2'] = [2,0,12,1,2,1,1,1,1]
priors1['c22r2'] = [2,0,12,1,8,1,1,1,1]
priors1['spaop'] = [2,0,16,1,6,1,1,1,1]
priors1['spinc'] = [2,0,12,0,8,1,1,1,1]
priors1['splan'] = [2,0,12,1,8,1,1,1,1]

priors2['mass'] = [2,0,10,1,2,1,1,1.79*10**18,1.48*10**18]
priors2['sma'] = [2,0,12,1,2,25657,91,1,1]
priors2['ecc'] = [2,0,1,0,1,1,1,1,1]
priors2['aop'] = [2,0,180,1,2,1,1,1,1]
priors2['inc'] = [2,0,180,1,2,1,1,1,1]
priors2['lan'] = [2,0,360,0,12,1,1,1,1]
priors2['mea'] = [2,0,180,1,2,1,1,1,1]
priors2['j2r2'] = [2,0,12,1,2,1,1,1,1]
priors2['c22r2'] = [2,0,12,1,8,1,1,1,1]
priors2['spaop'] = [2,0,16,1,6,1,1,1,1]
priors2['spinc'] = [2,0,12,0,8,1,1,1,1]
priors2['splan'] = [2,0,12,1,8,1,1,1,1]

def Not_mm_priors(priors, params):
    columnList = list(priors)
    totalLogProb = 0
    
    probDist = pd.DataFrame(columns = ['mass','sma','ecc','aop','inc','lan','mea','j2r2','c22r2','spaop','spinc','splan'],index=['PDF'])
   
    paramsList = params.to_numpy().tolist()
    count = 0
    allProbs = []
    numNaNs = 0
    #This loop runs through every column in the priors dataframe, and evaluates the probability density
    #function of the specified type.
    for i in columnList:
        count += 1
        
        #Uniform Distribution Shape
        if priors[i][0] == 0:
            for x in paramsList:
                if x[count] < priors[i][2] and x[count] > priors[i][1]:
                    allProbs.append(1)
                elif np.isnan(x[count]):
                    numNaNs += 1
                else:
                    allProbs.append(0)
        
        #Log-Uniform Distribution Shape
        elif priors[i][0] == 1:
            for x in paramsList:
                if x[count] < priors[i][4] and x[count] > priors[i][3]:
                    allProbs.append(1)
                elif np.isnan(x[count]):
                    numNaNs += 1
                else:
                    allProbs.append(0)
            
        # Normal Distribution Shape
        elif priors[i][0] == 2:
            for x in paramsList:
                if not np.isnan(x[count]):
                    allProbs.append(np.exp(-1/2*((x[count]-priors[i][6])/priors[i][5])**2))
            
        #Log Normal Distribution Shape
        elif priors[i][0] == 3:
            for x in paramsList:
                if not np.isnan(x[count]):
                    allProbs.append(np.exp(-1/2*(((np.log(x[count])-priors[i][8])**2)/(priors[i][7])**2))/x[count])
        else:
            print('Invalid input.') 
            
        #Here, add the Prior Probability Density function for this element to the total
    for x in allProbs:
        totalLogProb = totalLogProb + np.log(x)
        print(x)
        
    return totalLogProb

dist = Not_mm_priors(priorsP, paramsdf)
print('Total Log Probability: ', dist)